In [ ]:
# Cell 1: Install required libraries
# RUN THIS CELL FIRST TO INSTALL ALL DEPENDENCIES

!pip install torch torchvision torchaudio
!pip install transformers
!pip install ultralytics
!pip install opencv-python
!pip install pillow
!pip install matplotlib
!pip install numpy
!pip install kagglehub
!pip install scikit-learn
!pip install pyyaml
!pip install pandas

print("All required libraries installed successfully!")

In [ ]:
# Cell 2: Import all libraries
import torch
import torchvision
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import json
import os
import shutil
from sklearn.model_selection import train_test_split
import yaml
from pathlib import Path
import pandas as pd

print("All libraries imported successfully!")

All libraries imported successfully!


In [15]:
# Cell 3: Load pre-trained models (BASE MODELS)
print("Loading base models...")
yolo_model = YOLO('yolov8n.pt')
print("YOLO model loaded successfully!")

text_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
text_tokenizer.pad_token = text_tokenizer.eos_token
text_model = GPT2LMHeadModel.from_pretrained('gpt2')
print("GPT-2 model loaded successfully!")

Loading base models...
YOLO model loaded successfully!


c:\Users\RDHQ\Downloads\RADI-Image\.radi_image_venv\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT-2 model loaded successfully!


In [16]:
# Cell 4: Save base models
print("Saving base models...")
os.makedirs('../saved_models', exist_ok=True)
os.makedirs('../saved_models/text_generator', exist_ok=True)

torch.save(yolo_model.model.state_dict(), '../saved_models/yolo_model_base.pt')
text_model.save_pretrained('../saved_models/text_generator/')
text_tokenizer.save_pretrained('../saved_models/text_generator/')
print("Base models saved successfully!")

Saving base models...
Base models saved successfully!


In [17]:
# Cell 5: Define pipeline functions
def test_yolo_detection(image_path):
    """Test YOLO object detection"""
    results = yolo_model(image_path)
    detected_objects = []
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            class_id = int(box.cls[0])
            class_name = yolo_model.names[class_id]
            confidence = float(box.conf[0])
            detected_objects.append({
                'class': class_name,
                'confidence': confidence
            })
    
    return detected_objects

def generate_description(detected_objects):
    """Generate description based on detected objects"""
    if not detected_objects:
        return "No objects detected in the image."
    
    objects_list = ", ".join([obj['class'] for obj in detected_objects[:5]])
    prompt = f"In this image, I can see {objects_list}. This scene appears to be"
    
    inputs = text_tokenizer.encode(prompt, return_tensors='pt', max_length=512, truncation=True)
    
    with torch.no_grad():
        outputs = text_model.generate(
            inputs,
            max_length=150,
            num_return_sequences=1,
            temperature=0.7,
            pad_token_id=text_tokenizer.eos_token_id,
            do_sample=True
        )
    
    description = text_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

def test_pipeline(image_path=None):
    """Test the complete pipeline"""
    if image_path and os.path.exists(image_path):
        print(f"Processing image: {image_path}")
        
        detected_objects = test_yolo_detection(image_path)
        
        print("\nDetected objects:")
        for obj in detected_objects:
            print(f"  - {obj['class']}: {obj['confidence']:.3f}")
        
        description = generate_description(detected_objects)
        print(f"\nGenerated description: {description}")
        
        return detected_objects, description
    else:
        print("No valid image path provided")
        return [], ""

print("Pipeline functions defined!")

Pipeline functions defined!


In [18]:
# Cell 6: Test basic pipeline
print("Testing basic pipeline...")
# Example: test_pipeline('path_to_your_image.jpg')
print("Pipeline ready! Use test_pipeline('your_image_path') to test.")

Testing basic pipeline...
Pipeline ready! Use test_pipeline('your_image_path') to test.


In [19]:
# Cell 7: OPTIONAL - Download custom dataset
# UNCOMMENT THE FOLLOWING LINES TO DOWNLOAD CUSTOM DATASET

import kagglehub
print("Downloading dataset...")
path = kagglehub.dataset_download("junkie75/yolo-tiger-and-lion-labelled-detection")
print(f"Dataset downloaded to: {path}")

Dataset downloaded to: C:\Users\RDHQ\.cache\kagglehub\datasets\junkie75\yolo-tiger-and-lion-labelled-detection\versions\1


In [20]:
# Cell 8: OPTIONAL - Prepare dataset for YOLO training
# UNCOMMENT THE FOLLOWING LINES TO PREPARE DATASET

def prepare_yolo_dataset(dataset_path):
    """Prepare dataset in YOLO format"""
    
    dataset_dir = '../datasets/tiger_lion'
    for split in ['train', 'val', 'test']:
        os.makedirs(f'{dataset_dir}/images/{split}', exist_ok=True)
        os.makedirs(f'{dataset_dir}/labels/{split}', exist_ok=True)
    
    # Find all image files
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    image_files = []
    
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                image_files.append(os.path.join(root, file))
    
    print(f"Found {len(image_files)} images")
    
    # Split data
    train_files, temp_files = train_test_split(image_files, test_size=0.3, random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)
    
    # Copy files
    def copy_files(file_list, split_type):
        for img_path in file_list:
            label_path = img_path.replace('images', 'labels').rsplit('.', 1)[0] + '.txt'
            
            if os.path.exists(label_path):
                img_filename = os.path.basename(img_path)
                shutil.copy2(img_path, f'{dataset_dir}/images/{split_type}/{img_filename}')
                label_filename = os.path.basename(label_path)
                shutil.copy2(label_path, f'{dataset_dir}/labels/{split_type}/{label_filename}')
    
    copy_files(train_files, 'train')
    copy_files(val_files, 'val')
    copy_files(test_files, 'test')
    
    print(f"Dataset prepared: Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")
    return dataset_dir

# Prepare dataset (uncomment the line below when ready)
dataset_dir = prepare_yolo_dataset(path)

Found 951 images
Dataset prepared: Train: 665, Val: 143, Test: 143


In [21]:
# Cell 9: OPTIONAL - Create dataset.yaml configuration
# UNCOMMENT THE FOLLOWING LINES TO CREATE DATASET CONFIG

def create_dataset_yaml(dataset_dir):
    """Create dataset.yaml file for YOLO training"""
    dataset_config = {
        'path': os.path.abspath(dataset_dir),
        'train': 'images/train',
        'val': 'images/val', 
        'test': 'images/test',
        'nc': 2,
        'names': ['tiger', 'lion']
    }
    
    yaml_path = os.path.join(dataset_dir, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(dataset_config, f, default_flow_style=False)
    
    print(f"Dataset YAML created at: {yaml_path}")
    print(f"Configuration: {dataset_config}")
    return yaml_path

# Create dataset.yaml (uncomment the line below when ready)
yaml_path = create_dataset_yaml(dataset_dir)

Dataset YAML created at: ../datasets/tiger_lion\dataset.yaml
Configuration: {'path': 'c:\\Users\\RDHQ\\Downloads\\RADI-Image\\datasets\\tiger_lion', 'train': 'images/train', 'val': 'images/val', 'test': 'images/test', 'nc': 2, 'names': ['tiger', 'lion']}


In [22]:
# Cell 10: OPTIONAL - Setup training configuration
# UNCOMMENT THE FOLLOWING LINES TO SETUP TRAINING

print("Setting up training configuration...")

# Check if dataset.yaml exists
yaml_path = '../datasets/tiger_lion/dataset.yaml'
if not os.path.exists(yaml_path):
    print(f"ERROR: Dataset YAML not found at {yaml_path}")
    print("Please run Cells 7, 8, and 9 first to download and prepare the dataset.")
else:
    training_args = {
        'data': yaml_path,
        'epochs': 3,
        'imgsz': 640,
        'batch': 8,
        'patience': 5,
        'save': True,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'workers': 2,
        'lr0': 0.01,
        'project': '../runs/detect',
        'name': 'tiger_lion_detection',
        'exist_ok': True
    }

    print(f"Training configuration ready! Device: {training_args['device']}")
    print(f"Using dataset: {yaml_path}")

Setting up training configuration...
Training configuration ready! Device: cpu
Using dataset: ../datasets/tiger_lion/dataset.yaml


In [23]:
# Cell 11: OPTIONAL - Start training
# UNCOMMENT THE FOLLOWING LINES TO START TRAINING

print("Loading model for training...")
train_model = YOLO('yolov8n.pt')  # Separate model for training

print("Starting training...")
results = train_model.train(**training_args)
print("Training completed!")

Loading model for training...
Starting training...


New https://pypi.org/project/ultralytics/8.3.214 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.200  Python-3.11.9 torch-2.0.1+cpu CPU (Intel Core(TM) i7-7700 3.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=../datasets/tiger_lion/dataset.yaml, epochs=3, patience=5, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=2, project=../runs/detect, name=tiger_lion_detection, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, 

Training completed!


In [30]:
# Cell 12: OPTIONAL - Save trained model (SIMPLIFIED)
# UNCOMMENT THE FOLLOWING LINES TO SAVE TRAINED MODEL

print("Saving trained model...")

# YOLO automatically saves the best model during training, so we don't need to export
# Just verify that the training files exist
best_model_path = '../runs/detect/tiger_lion_detection/weights/best.pt'
last_model_path = '../runs/detect/tiger_lion_detection/weights/last.pt'

if os.path.exists(best_model_path):
    print(f"Best model already saved at: {best_model_path}")
    # Copy to our saved_models directory for easier access
    shutil.copy(best_model_path, '../saved_models/yolo_model_trained.pt')
    print("Trained model copied to saved_models directory!")
else:
    print("No best.pt found. Training might have failed or not completed.")

Saving trained model...
Best model already saved at: ../runs/detect/tiger_lion_detection/weights/best.pt
Trained model copied to saved_models directory!


In [31]:
# Cell 13: OPTIONAL - Load trained model (SIMPLIFIED)
# UNCOMMENT THE FOLLOWING LINES TO LOAD TRAINED MODEL

print("Loading trained model...")

# Check for trained model in multiple locations
trained_model_path = None

# First check our saved_models directory
if os.path.exists('../saved_models/yolo_model_trained.pt'):
    trained_model_path = '../saved_models/yolo_model_trained.pt'
    print("Found trained model in saved_models directory")

# Then check the training runs
elif os.path.exists('../runs/detect/tiger_lion_detection/weights/best.pt'):
    trained_model_path = '../runs/detect/tiger_lion_detection/weights/best.pt'
    print("Found trained model in runs directory")

elif os.path.exists('../runs/detect/tiger_lion_detection/weights/last.pt'):
    trained_model_path = '../runs/detect/tiger_lion_detection/weights/last.pt'
    print("Found trained model (last.pt) in runs directory")

# Check for torchscript version (what was actually created)
elif os.path.exists('../runs/detect/tiger_lion_detection/weights/best.torchscript'):
    print("Found torchscript model, but need .pt format for YOLO class")
    # We can't directly load torchscript with YOLO class, so use the original best.pt
    if os.path.exists('../runs/detect/tiger_lion_detection/weights/best.pt'):
        trained_model_path = '../runs/detect/tiger_lion_detection/weights/best.pt'
    else:
        print("No .pt format found for torchscript model")

if trained_model_path:
    yolo_model = YOLO(trained_model_path)
    print(f"Trained model loaded successfully from: {trained_model_path}")
else:
    print("No trained model found. Using base YOLO model.")
    print("Available files in runs directory:")
    runs_dir = '../runs/detect/tiger_lion_detection/weights'
    if os.path.exists(runs_dir):
        for file in os.listdir(runs_dir):
            print(f"  - {file}")

Loading trained model...
Found trained model in saved_models directory
Trained model loaded successfully from: ../saved_models/yolo_model_trained.pt


In [ ]:
# Cell 14: OPTIONAL - Visualize training results (FIXED FOR SPACES IN COLUMNS)
# UNCOMMENT THE FOLLOWING LINES FOR VISUALIZATION

def plot_training_results_fixed():
    """Plot training metrics - fixed for column names with spaces"""
    try:
        results_file = '../runs/detect/tiger_lion_detection/results.csv'
        if os.path.exists(results_file):
            df = pd.read_csv(results_file)
            
            # Clean column names by stripping whitespace
            df.columns = df.columns.str.strip()
            print("Cleaned column names:")
            for col in df.columns:
                print(f"  - '{col}'")
            
            # Create figure with subplots
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
            fig.suptitle('Training Metrics (3 Epochs)', fontsize=16, fontweight='bold')
            
            # Plot 1: Precision & Recall
            ax1.plot(df['epoch'], df['metrics/precision(B)'], 'b-', label='Precision', linewidth=2, marker='o')
            ax1.plot(df['epoch'], df['metrics/recall(B)'], 'r-', label='Recall', linewidth=2, marker='s')
            ax1.set_title('Precision & Recall')
            ax1.set_xlabel('Epoch')
            ax1.set_ylabel('Score')
            ax1.legend()
            ax1.grid(True, alpha=0.3)
            
            # Plot 2: mAP Metrics
            ax2.plot(df['epoch'], df['metrics/mAP50(B)'], 'g-', label='mAP50', linewidth=2, marker='o')
            ax2.plot(df['epoch'], df['metrics/mAP50-95(B)'], 'orange', label='mAP50-95', linewidth=2, marker='s')
            ax2.set_title('mAP Metrics')
            ax2.set_xlabel('Epoch')
            ax2.set_ylabel('mAP Score')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            
            # Plot 3: Box Loss
            ax3.plot(df['epoch'], df['train/box_loss'], 'b-', label='Train Box Loss', linewidth=2, marker='o')
            ax3.plot(df['epoch'], df['val/box_loss'], 'r-', label='Val Box Loss', linewidth=2, marker='s')
            ax3.set_title('Box Loss')
            ax3.set_xlabel('Epoch')
            ax3.set_ylabel('Loss')
            ax3.legend()
            ax3.grid(True, alpha=0.3)
            
            # Plot 4: Classification Loss
            ax4.plot(df['epoch'], df['train/cls_loss'], 'b-', label='Train Cls Loss', linewidth=2, marker='o')
            ax4.plot(df['epoch'], df['val/cls_loss'], 'r-', label='Val Cls Loss', linewidth=2, marker='s')
            ax4.set_title('Classification Loss')
            ax4.set_xlabel('Epoch')
            ax4.set_ylabel('Loss')
            ax4.legend()
            ax4.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            # Print training summary
            print("\n" + "="*50)
            print("TRAINING SUMMARY (3 Epochs)")
            print("="*50)
            print(f"Final Precision: {df['metrics/precision(B)'].iloc[-1]:.4f}")
            print(f"Final Recall: {df['metrics/recall(B)'].iloc[-1]:.4f}")
            print(f"Final mAP50: {df['metrics/mAP50(B)'].iloc[-1]:.4f}")
            print(f"Final mAP50-95: {df['metrics/mAP50-95(B)'].iloc[-1]:.4f}")
            print(f"Final Train Box Loss: {df['train/box_loss'].iloc[-1]:.4f}")
            print(f"Final Val Box Loss: {df['val/box_loss'].iloc[-1]:.4f}")
            
            # Show improvement from first to last epoch
            if len(df) > 1:
                print("\nImprovement from Epoch 1 to 3:")
                precision_improve = df['metrics/precision(B)'].iloc[-1] - df['metrics/precision(B)'].iloc[0]
                recall_improve = df['metrics/recall(B)'].iloc[-1] - df['metrics/recall(B)'].iloc[0]
                map50_improve = df['metrics/mAP50(B)'].iloc[-1] - df['metrics/mAP50(B)'].iloc[0]
                print(f"Precision: {precision_improve:+.4f}")
                print(f"Recall: {recall_improve:+.4f}")
                print(f"mAP50: {map50_improve:+.4f}")
            
        else:
            print(f"Results file not found at: {results_file}")
            
    except Exception as e:
        print(f"Error plotting training results: {e}")
        import traceback
        traceback.print_exc()

plot_training_results_fixed()

Cleaned column names:
  - 'epoch'
  - 'train/box_loss'
  - 'train/cls_loss'
  - 'train/dfl_loss'
  - 'metrics/precision(B)'
  - 'metrics/recall(B)'
  - 'metrics/mAP50(B)'
  - 'metrics/mAP50-95(B)'
  - 'val/box_loss'
  - 'val/cls_loss'
  - 'val/dfl_loss'
  - 'lr/pg0'
  - 'lr/pg1'
  - 'lr/pg2'


<Figure size 1200x800 with 4 Axes>

<Figure size 1500x1000 with 2 Axes>

<Figure size 1200x800 with 4 Axes>


TRAINING SUMMARY (3 Epochs)
Final Precision: 0.5555
Final Recall: 0.5948
Final mAP50: 0.5645
Final mAP50-95: 0.2801
Final Train Box Loss: 1.1025
Final Val Box Loss: 1.4007

Improvement from Epoch 1 to 3:
Precision: +0.1306
Recall: +0.3518
mAP50: +0.3445
